In [1]:
import numpy as np
from casadi import *
import casadi as cs
import matplotlib.pyplot as plt
import seaborn as sns
from util import *
import itertools 
from time import perf_counter
import sympy as sym
from IPython.display import display, Math


In [2]:
from dpilqr import util

In [3]:
%load_ext autoreload
%autoreload 2

In [49]:
theta_max = np.pi / 6
phi_max = np.pi / 6

v_max = 3
v_min = -3

theta_min = -np.pi / 6
phi_min = -np.pi / 6

tau_max = 15
tau_min = 0

x_min = -5
x_max = 5

y_min = -5
y_max = 5

z_min = 0
z_max = 3.5

max_input_base = np.array([[theta_max], [phi_max], [tau_max]])
min_input_base = np.array([[theta_min], [phi_min], [tau_min]])
max_state_base = np.array([[x_max], [y_max], [z_max], [v_max],[v_max], [v_max]])
min_state_base = np.array([[x_min], [y_min], [z_min], [v_min],[v_min],[v_min]])


In [53]:
n_agents = 3

In [54]:

max_input = np.tile(max_input_base,(n_agents,1))
min_input = np.tile(min_input_base,n_agents)
max_state = np.tile(max_state_base,n_agents)
min_state = np.tile(min_state_base,n_agents)

In [57]:
max_input

array([[ 0.524],
       [ 0.524],
       [15.   ],
       [ 0.524],
       [ 0.524],
       [15.   ],
       [ 0.524],
       [ 0.524],
       [15.   ]])

In [51]:
max_input = np.array([[theta_max], [phi_max], [tau_max], \
                          [theta_max], [phi_max], [tau_max], \
                          [theta_max], [phi_max], [tau_max]])

In [52]:
max_input

array([[ 0.524],
       [ 0.524],
       [15.   ],
       [ 0.524],
       [ 0.524],
       [15.   ],
       [ 0.524],
       [ 0.524],
       [15.   ]])